In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC10_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 4
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC10(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC10(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC10(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料
Progress: 100% - Completed
讀取3秒資料


D:\Hu_code\ResNet18_ESC10\ESC10_kFold_531s\..\TrainDataPreprocessing.py:64: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
Data Loading Time: 26.22 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=10)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        
        #------------------------------------------------
        #需要調整的地方
        #------------------------------------------------
        #--要用幾秒訓練 data_1S/data_3S/data_1S
        #--學習率
        #--epoch
        if(TL==0):
            trainData = data_5S
            epoch = 90
            lr = 0.0032
        elif(TL==1):
            trainData = data_3S
            epoch = 90
            lr = 0.0032
        else:
            trainData = data_1S
            epoch = 90
            lr = 0.0032
        #------------------------------------------------
    

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型
        model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold,
                                                                setLR=lr,
                                                                setEpoch=epoch)
    
        #測試準確度
        test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
        test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
        test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/90 Train_Loss: 1.6476500034332275 Train_acc: 20.625000%
Train Epoch: 2/90 Train_Loss: 1.234646201133728 Train_acc: 45.937500%
Train Epoch: 3/90 Train_Loss: 1.0444834232330322 Train_acc: 55.625000%
Train Epoch: 4/90 Train_Loss: 0.9221799373626709 Train_acc: 60.000000%
Train Epoch: 5/90 Train_Loss: 0.978413462638855 Train_acc: 70.000000%
Train Epoch: 6/90 Train_Loss: 0.6573153734207153 Train_acc: 72.187500%
Train Epoch: 7/90 Train_Loss: 0.7104108929634094 Train_acc: 78.437500%
Train Epoch: 8/90 Train_Loss: 0.5802449584007263 Train_acc: 82.187500%
Train Epoch: 9/90 Train_Loss: 0.47712838649749756 Train_acc: 80.625000%
Train Epoch: 10/90 Train_Loss: 0.3751607835292816 Train_acc: 82.812500%
Train Epoch: 11/90 Train_Loss: 0.7

Train Epoch: 15/90 Train_Loss: 0.1634964793920517 Train_acc: 94.791672%
Train Epoch: 16/90 Train_Loss: 0.05897917598485947 Train_acc: 95.520836%
Train Epoch: 17/90 Train_Loss: 0.12509959936141968 Train_acc: 95.937508%
Train Epoch: 18/90 Train_Loss: 0.03779866546392441 Train_acc: 97.187508%
Train Epoch: 19/90 Train_Loss: 0.038757577538490295 Train_acc: 97.812508%
Train Epoch: 20/90 Train_Loss: 0.12271136045455933 Train_acc: 97.708336%
Train Epoch: 21/90 Train_Loss: 0.0530201718211174 Train_acc: 96.770836%
Train Epoch: 22/90 Train_Loss: 0.09340596199035645 Train_acc: 97.604172%
Train Epoch: 23/90 Train_Loss: 0.09649960696697235 Train_acc: 97.604172%
Train Epoch: 24/90 Train_Loss: 0.03815564140677452 Train_acc: 97.708336%
Train Epoch: 25/90 Train_Loss: 0.047254160046577454 Train_acc: 97.708336%
Train Epoch: 26/90 Train_Loss: 0.024346617981791496 Train_acc: 97.083336%
Train Epoch: 27/90 Train_Loss: 0.026834914460778236 Train_acc: 97.083336%
Train Epoch: 28/90 Train_Loss: 0.2102814912796020

Train Epoch: 32/90 Train_Loss: 0.09410259872674942 Train_acc: 93.750000%
Train Epoch: 33/90 Train_Loss: 0.09304179251194 Train_acc: 94.625000%
Train Epoch: 34/90 Train_Loss: 0.149183452129364 Train_acc: 94.875000%
Train Epoch: 35/90 Train_Loss: 0.1209506168961525 Train_acc: 94.125000%
Train Epoch: 36/90 Train_Loss: 0.10898713022470474 Train_acc: 95.062500%
Train Epoch: 37/90 Train_Loss: 0.05203919857740402 Train_acc: 94.687500%
Train Epoch: 38/90 Train_Loss: 0.20815323293209076 Train_acc: 94.750000%
Train Epoch: 39/90 Train_Loss: 0.13689963519573212 Train_acc: 95.125000%
Train Epoch: 40/90 Train_Loss: 0.24614451825618744 Train_acc: 94.687500%
Train Epoch: 41/90 Train_Loss: 0.09312465041875839 Train_acc: 94.937500%
Train Epoch: 42/90 Train_Loss: 0.05937264487147331 Train_acc: 95.187500%
Train Epoch: 43/90 Train_Loss: 0.11208498477935791 Train_acc: 94.812500%
Train Epoch: 44/90 Train_Loss: 0.10787409543991089 Train_acc: 95.062500%
Train Epoch: 45/90 Train_Loss: 0.15126769244670868 Train_

Train Epoch: 50/90 Train_Loss: 0.01536682527512312 Train_acc: 99.375000%
Train Epoch: 51/90 Train_Loss: 0.01537586934864521 Train_acc: 99.687500%
Train Epoch: 52/90 Train_Loss: 0.008814127184450626 Train_acc: 100.000000%
Train Epoch: 53/90 Train_Loss: 0.005599430296570063 Train_acc: 100.000000%
Train Epoch: 54/90 Train_Loss: 0.035227417945861816 Train_acc: 99.687500%
Train Epoch: 55/90 Train_Loss: 0.004302187357097864 Train_acc: 99.687500%
Train Epoch: 56/90 Train_Loss: 0.003869229694828391 Train_acc: 99.375000%
Train Epoch: 57/90 Train_Loss: 0.023706987500190735 Train_acc: 100.000000%
Train Epoch: 58/90 Train_Loss: 0.026650376617908478 Train_acc: 99.687500%
Train Epoch: 59/90 Train_Loss: 0.006262732204049826 Train_acc: 100.000000%
Train Epoch: 60/90 Train_Loss: 0.004454334732145071 Train_acc: 100.000000%
Train Epoch: 61/90 Train_Loss: 0.006685225758701563 Train_acc: 100.000000%
Train Epoch: 62/90 Train_Loss: 0.0023156721144914627 Train_acc: 100.000000%
Train Epoch: 63/90 Train_Loss: 0

Train Epoch: 66/90 Train_Loss: 0.037247221916913986 Train_acc: 98.750008%
Train Epoch: 67/90 Train_Loss: 0.001950136385858059 Train_acc: 98.854172%
Train Epoch: 68/90 Train_Loss: 0.02033369615674019 Train_acc: 98.854172%
Train Epoch: 69/90 Train_Loss: 0.002332223579287529 Train_acc: 98.854172%
Train Epoch: 70/90 Train_Loss: 0.03609893098473549 Train_acc: 98.854172%
Train Epoch: 71/90 Train_Loss: 0.024893028661608696 Train_acc: 98.854172%
Train Epoch: 72/90 Train_Loss: 0.024970445781946182 Train_acc: 98.854172%
Train Epoch: 73/90 Train_Loss: 0.02430044114589691 Train_acc: 98.958336%
Train Epoch: 74/90 Train_Loss: 0.0004084338725078851 Train_acc: 98.750008%
Train Epoch: 75/90 Train_Loss: 0.021623821929097176 Train_acc: 98.854172%
Train Epoch: 76/90 Train_Loss: 0.04519704356789589 Train_acc: 98.854172%
Train Epoch: 77/90 Train_Loss: 0.011608632281422615 Train_acc: 98.854172%
Train Epoch: 78/90 Train_Loss: 0.013717184774577618 Train_acc: 98.958336%
Train Epoch: 79/90 Train_Loss: 0.01274747

...

Train Epoch: 40/90 Train_Loss: 0.0532069131731987 Train_acc: 98.125000%
Train Epoch: 41/90 Train_Loss: 0.07038760185241699 Train_acc: 97.500000%
Train Epoch: 42/90 Train_Loss: 0.07244426757097244 Train_acc: 98.437500%
Train Epoch: 43/90 Train_Loss: 0.008902630768716335 Train_acc: 100.000000%
Train Epoch: 44/90 Train_Loss: 0.04402606934309006 Train_acc: 99.375000%
Train Epoch: 45/90 Train_Loss: 0.012420384213328362 Train_acc: 99.062500%
Train Epoch: 46/90 Train_Loss: 0.013159569352865219 Train_acc: 99.687500%
Train Epoch: 47/90 Train_Loss: 0.04126898571848869 Train_acc: 99.375000%
Train Epoch: 48/90 Train_Loss: 0.015115157701075077 Train_acc: 100.000000%
Train Epoch: 49/90 Train_Loss: 0.009398403577506542 Train_acc: 99.687500%
Train Epoch: 50/90 Train_Loss: 0.005506876390427351 Train_acc: 100.000000%
Train Epoch: 51/90 Train_Loss: 0.013303776271641254 Train_acc: 100.000000%
Train Epoch: 52/90 Train_Loss: 0.006378646939992905 Train_acc: 100.000000%
Train Epoch: 53/90 Train_Loss: 0.002250

Train Epoch: 56/90 Train_Loss: 0.06134321540594101 Train_acc: 98.541672%
Train Epoch: 57/90 Train_Loss: 0.0012659168569371104 Train_acc: 98.645836%
Train Epoch: 58/90 Train_Loss: 0.03815441578626633 Train_acc: 98.645836%
Train Epoch: 59/90 Train_Loss: 0.022841189056634903 Train_acc: 98.645836%
Train Epoch: 60/90 Train_Loss: 0.02486512064933777 Train_acc: 98.541672%
Train Epoch: 61/90 Train_Loss: 0.010913162492215633 Train_acc: 98.645836%
Train Epoch: 62/90 Train_Loss: 0.019739070907235146 Train_acc: 98.645836%
Train Epoch: 63/90 Train_Loss: 0.02898627705872059 Train_acc: 98.645836%
Train Epoch: 64/90 Train_Loss: 0.0007702290895394981 Train_acc: 98.645836%
Train Epoch: 65/90 Train_Loss: 0.01210495363920927 Train_acc: 98.645836%
Train Epoch: 66/90 Train_Loss: 0.018211113288998604 Train_acc: 98.645836%
Train Epoch: 67/90 Train_Loss: 0.05524022504687309 Train_acc: 98.645836%
Train Epoch: 68/90 Train_Loss: 0.0008706972002983093 Train_acc: 98.645836%
Train Epoch: 69/90 Train_Loss: 0.02910118

Train Epoch: 73/90 Train_Loss: 0.0628160759806633 Train_acc: 94.875000%
Train Epoch: 74/90 Train_Loss: 0.05025395378470421 Train_acc: 94.812500%
Train Epoch: 75/90 Train_Loss: 0.14641311764717102 Train_acc: 94.750000%
Train Epoch: 76/90 Train_Loss: 0.11912251263856888 Train_acc: 94.937500%
Train Epoch: 77/90 Train_Loss: 0.05796603858470917 Train_acc: 95.062500%
Train Epoch: 78/90 Train_Loss: 0.04497630521655083 Train_acc: 95.062500%
Train Epoch: 79/90 Train_Loss: 0.09182530641555786 Train_acc: 95.062500%
Train Epoch: 80/90 Train_Loss: 0.1269538849592209 Train_acc: 95.062500%
Train Epoch: 81/90 Train_Loss: 0.08540341258049011 Train_acc: 95.125000%
Train Epoch: 82/90 Train_Loss: 0.0832773745059967 Train_acc: 95.062500%
Train Epoch: 83/90 Train_Loss: 0.030780401080846786 Train_acc: 94.750000%
Train Epoch: 84/90 Train_Loss: 0.19304363429546356 Train_acc: 95.125000%
Train Epoch: 85/90 Train_Loss: 0.1655067503452301 Train_acc: 94.687500%
Train Epoch: 86/90 Train_Loss: 0.17502324283123016 Tra

Train Epoch: 90/90 Train_Loss: 0.00998907070606947 Train_acc: 99.687500%
Test accuracy of the model: 54.000000%
Test accuracy of the model: 74.583333%
Test accuracy of the model: 82.500000%
Training Time: 31.47 seconds
----------------------------------------------------------------------------------------------------------------
fold:  5 TL:  1
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/90 Train_Loss: 0.6486186385154724 Train_acc: 65.833336%
Train Epoch: 2/90 Train_Loss: 1.2005599737167358 Train_acc: 72.916672%
Train Epoch: 3/90 Train_Loss: 0.7264606952667236 Train_acc: 80.104172%
Train Epoch: 4/90 Train_Loss: 0.41233959794044495 Train_acc: 83.333336%
Train Epoch: 5/90 Train_Loss: 0.48819661140441895 Train_acc: 84.583336%
Train Epoch: 6/90 Train_Loss: 0.425956666469574 Train_acc: 85.833336%
Train Epoch: 7/90 Train_Loss: 0.5264127254486084 Train_acc: 86.562508%
Train Epoch: 8/90 Train_Loss: 0.579310894

Train Epoch: 12/90 Train_Loss: 0.22356949746608734 Train_acc: 91.250000%
Train Epoch: 13/90 Train_Loss: 0.1317867785692215 Train_acc: 91.000000%
Train Epoch: 14/90 Train_Loss: 0.23586885631084442 Train_acc: 90.562500%
Train Epoch: 15/90 Train_Loss: 0.14931684732437134 Train_acc: 90.187500%
Train Epoch: 16/90 Train_Loss: 0.17762577533721924 Train_acc: 91.625000%
Train Epoch: 17/90 Train_Loss: 0.20002231001853943 Train_acc: 92.187500%
Train Epoch: 18/90 Train_Loss: 0.20954130589962006 Train_acc: 92.750000%
Train Epoch: 19/90 Train_Loss: 0.11676964163780212 Train_acc: 92.625000%
Train Epoch: 20/90 Train_Loss: 0.14835749566555023 Train_acc: 92.750000%
Train Epoch: 21/90 Train_Loss: 0.20821063220500946 Train_acc: 92.875000%
Train Epoch: 22/90 Train_Loss: 0.2580326795578003 Train_acc: 93.187500%
Train Epoch: 23/90 Train_Loss: 0.09593790769577026 Train_acc: 93.875000%
Train Epoch: 24/90 Train_Loss: 0.27496638894081116 Train_acc: 93.625000%
Train Epoch: 25/90 Train_Loss: 0.12333658337593079 Tr

In [5]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()